In [1]:
import pandas as pd
import numpy as np

# Cleaning up the dataset

First we'll load in the dataset and take a look at it to determine what needs to be done.

In [15]:
#read in the dataframe
df = pd.read_csv('data/gardner_mt_catastrophe_only_tubulin.csv', header=None)

df.head()

,0,1,2,3,4,5
0,# Raw data from Gardner,et al.,Cell,147,1092,2011.0
1,#,NaN,NaN,NaN,NaN,NaN
2,# Each entry in the respective columns is the ...,in seconds,,NaN,NaN,NaN
3,# between the start of growth of a microtubule...,NaN,NaN,NaN,NaN,NaN
4,#,NaN,NaN,NaN,NaN,NaN


If we look at the ```dataframe```, we can see that the metadata has been read in to the first few rows. Since we need this information to be separate from the data, we will write a function that extracts the metadata, and then remove these rows from the ```dataframe```. 

In [3]:
def metadata_reader(df):
    '''
    manually extract the metadata from the first few rows of the data frame df.
    '''
    
    #make a dictionary that will be filled with the relevant metadata
    meta_dict = {}
    
    #find author and update dictionary
    author_cell = df.loc[0, 0]
    author = author_cell[author_cell.find('from')+4 :].strip()
    meta_dict.update({'author':author})
    
    #find journal info and update dictionary
    journal = str(df.loc[0, 2])
    issue   = str(int(df.loc[0, 3]))
    version = str(int(df.loc[0, 4]))
    year    = str(int(df.loc[0, 5]))
    citation = journal + ', ' + issue + ', ' + version + ', ' + year
    meta_dict.update({'citation':citation})
    
    #find acknowledgement and update dictionary
    acknowledgement = df.loc[8, 0]
    acknowledgement = acknowledgement.strip('# ')
    meta_dict.update({'acknowledgement':acknowledgement})
    
    #find institution and update dictionary
    institution = df.loc[8, 1]
    meta_dict.update({'institution':institution})
    
    return meta_dict

In [4]:
# extract the metadata
metadata = metadata_reader(df)

print(metadata)

{'author': 'Gardner', 'citation': ' Cell, 147, 1092, 2011', 'acknowledgement': 'Kindly provided by Melissa Gardner', 'institution': ' University of Minnesota'}


Now we will drop the first 9 rows containing the metadata. 

In [5]:
#drop the first 9 rows
df = df.drop(np.arange(9))

#check that we got it
df.head()

,0,1,2,3,4,5
9,12 uM,7 uM,9 uM,10 uM,14 uM,NaN
10,25,35,25,50,60,NaN
11,40,45,40,60,75,NaN
12,40,50,40,60,75,NaN
13,45.429,50,45,75,85,NaN


While we have cleaned up the data, the dataset is not yet tidy. The first row contains text that should be the column names, and the 5th column contains only NaN values.

We will remove the irrelevant column, then write and use a function that will set the names in row 9 as the column names.

In [6]:
# get rid of the 5th column
del df[5]

# check the dataframe
df.head()

,0,1,2,3,4
9,12 uM,7 uM,9 uM,10 uM,14 uM
10,25,35,25,50,60
11,40,45,40,60,75
12,40,50,40,60,75
13,45.429,50,45,75,85


In [7]:
def column_renamer_first_row(df):
    '''
    Renames the columns of the dataframe with the names of the first row, then deletes 1st row.
    Resets indices.
    '''
    
    #reset indices so we know the first row is 0
    #need to do drop=True, otherwise you will get the first column named 'index'
    df = df.reset_index(drop=True)
    
    #make an empty dict to fill with the column names for renaming purposes
    column_dict = {}
    
    #go through the first row and make all the replacement column names
    for i, element in enumerate(df.columns):
    
        replacement = df.loc[0,i]
        
        column_dict.update({element:replacement})
        
    #now rename it based on the replacement dict
    df = df.rename(columns=column_dict)
    
    #drop the first row, which is redundant now
    df = df.drop(0)
    
    return df

In [8]:
#use the new function to rename the columns
df_renamed = column_renamer_first_row(df)

#check that we got it
df_renamed.tail()

,12 uM,7 uM,9 uM,10 uM,14 uM
688,1335,NaN,NaN,NaN,NaN
689,1485,NaN,NaN,NaN,NaN
690,1505,NaN,NaN,NaN,NaN
691,1520,NaN,NaN,NaN,NaN
692,1785,NaN,NaN,NaN,NaN


Now we will reorder the columns in order of increasing concentration, then melt the dataframe such that there will be a column to indicate the tubulin concentration, and one to indicate the time to catastrophe recorded.

In [9]:
#change the order of the columns
col_list = ['7 uM', '9 uM', '10 uM', '12 uM', '14 uM']
df_renamed = df_renamed.reindex(columns = col_list)

#check that we got it
df_renamed.head()

,7 uM,9 uM,10 uM,12 uM,14 uM
1,35,25,50,25,60
2,45,40,60,40,75
3,50,40,60,40,75
4,50,45,75,45.429,85
5,55,50,75,50,115


In [10]:
# melt it down
df_melted = pd.melt(df_renamed, var_name='concentration', value_name='time to catastrophe (s)')

df_melted.tail()

,concentration,time to catastrophe (s)
3455,14 uM,NaN
3456,14 uM,NaN
3457,14 uM,NaN
3458,14 uM,NaN
3459,14 uM,NaN


The dataframe is now tidy, however, there are many NaN values. These will not be useful to our modelling, so we will write a function to remove these rows.

In [11]:
def is_nan(x):
    '''
    Returns True if x is a Nan.
    '''
    
    boo = False
    
    if float(x) != float(x):
        boo = True
        
    return boo


def nan_buster(df, col_name, reset_index=False):
    '''
    1) Removes all of the rows in df that have NaNs in the specified column
    2) Can reset the indices if desired
    '''
    
    for i, element in enumerate(list(df[col_name])):
        
        if is_nan(element):
            
            df = df.drop(i)
            
    if reset_index:
        
        df = df.reset_index()
        
    return df

In [12]:
df_final = nan_buster(df_melted, 'time to catastrophe (s)', reset_index=True)
df_final = df_final.drop(columns=['index'])

df_final.tail()

,concentration,time to catastrophe (s)
1915,14 uM,1005
1916,14 uM,1135
1917,14 uM,1305
1918,14 uM,1400
1919,14 uM,1420


Now we have a tidy dataframe with no NaNs. 

The last step will be to convert every time to catastrophe into a float, since they are currently strings.

In [13]:
#go through every entry and convert everything in the time category to a float
for i in range(len(df_final)):
    
    value = df_final.loc[i, 'time to catastrophe (s)']
    
    df_final.loc[i, 'time to catastrophe (s)'] = float(value)

#check that we got it
print(type(df_final.loc[100, 'time to catastrophe (s)']))

<class 'float'>


Now we'll export the data to use in our analysis

In [16]:
df_final.to_csv('data/gardner_mt_catastrophe_only_tubulin_tidy.csv', index=False)